In [131]:
import backtrader as bt
from backtester import backtest15,backtestday,backtest1hr
import numpy as np
                    
import pandas as pd

import logging
logging.basicConfig(filename="log.txt", level=logging.DEBUG)

In [133]:
pnl = np.array([])

In [134]:
# pnl=np.array([])
class Nimisha(bt.Strategy):
    
    def __init__(self):
        self.lookback=15
        
    def log(self, txt):
        date = self.data.datetime.date(0)
        time = self.data.datetime.time()
        logging.debug('%s-%s, %s' % (date.isoformat(), time, txt))

#     def get_pnl(self):
#         return self.pnl

    def notify_order(self, order):

        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log("Buy EXECUTED {}".format(order.executed.price))
            elif order.issell():
                self.log("Sell EXECUTED {}".format(order.executed.price))

    def next(self):
        ohlc1 = (self.data.close[-1]+self.data.high[-1]+self.data.low[-1]+self.data.open[-1])/4
        ohlc2 = (self.data.close[-2]+self.data.high[-2]+self.data.low[-2]+self.data.open[-2])/4
        ohlc3 = (self.data.close[-3]+self.data.high[-3]+self.data.low[-3]+self.data.open[-3])/4
        ohlc = self.data.close.get(size=self.lookback,ago=-1)+self.data.high.get(size=self.lookback,ago=-1)+self.data.low.get(size=self.lookback,ago=-1)+self.data.open.get(size=self.lookback,ago=-1)
        ohlc = np.array(ohlc)
        ohlc/=4
        std = np.array(ohlc).std()
        cond1 = ohlc1>ohlc2 and ohlc2>ohlc3
        cond2 = (ohlc1>ohlc2+std/2)
        cond3 = self.data.high[-1]<self.data.close[0]
        lcond = (cond1 or cond2) and cond3
        if(lcond):
            if(self.data.high[0]>self.data.open[0]*1.005):
                pnl = np.append(pnl,self.data.open[0]*0.005)
            else:
                pnl = np.append(pnl,self.data.close[0]-self.data.open[0])

In [136]:
def analysis(data):
    pnl=np.array([])
    nim = Nimisha()
    try:
        backtest1hr(data, nim)
    except:
        pass
#     pnl = np.array(pnl)

#     aa = nim.get_pnl()
    aa = pnl
    return aa.sum(), len(aa),len(aa[aa>0]),len(aa[aa<0])

In [137]:
analysis('RELIANCE')

Starting Portfolio Value: 100000.00


(0.0, 0, 0, 0)

IndentationError: expected an indented block (1763208468.py, line 3)

In [50]:
pnl.sum()

2995.8052500000003

In [51]:
pnl

array([  4.51   ,   4.53425,   0.35   ,   4.57375,   4.6225 ,   4.486  ,
         4.4825 ,   2.95   ,  -0.6    ,   4.4775 ,   4.451  ,   4.474  ,
         4.5235 ,   4.54   ,   0.6    ,   1.85   ,   2.3    ,   2.95   ,
         4.7    ,   4.65   ,   4.67125,   4.725  ,   4.81575,   4.99525,
        -1.4    ,  -2.65   ,   4.86525,   4.897  ,   4.91525,  -0.8    ,
        -2.55   ,   4.5715 ,   4.6    ,   4.6255 ,   4.667  ,   3.     ,
         2.65   ,   4.55   ,   4.8675 ,   4.9355 ,   2.5    ,   2.15   ,
         4.9945 ,   3.05   ,   4.9765 ,   5.03   ,   5.007  ,   3.95   ,
         3.65   ,   4.806  ,   4.835  ,   4.859  ,   1.15   ,   4.92475,
         4.985  ,   5.01375,   3.9    ,   2.8    ,   5.1665 ,   5.22175,
         4.15   ,   5.32475,   5.44125,   5.4815 ,   5.42775,   5.492  ,
         5.4665 ,   5.567  ,   5.58075,   2.35   ,   5.65   ,   5.69925,
         5.755  ,   5.82325,   5.889  ,   5.952  ,   5.8835 ,   5.90575,
         3.9    ,   5.9685 ,   4.7    ,   0.25   , 

In [77]:
a = np.array([])
a = np.append(a,3)
a

array([3.])

In [74]:
a.append(3)

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [187]:
def analysis(symbol):
    data = pd.read_csv("1hr/"+symbol+".csv")
    lookback=14
    sma = 40
    data['ohlc'] = (data.High+data.Low+data.Close+data.Open)/4
    pnl=[]
    for i in range(sma+5,len(data)):
        std = np.array(data.ohlc[i-lookback:i]).std()
        ohlc1,ohlc2,ohlc3 = data.ohlc[i-1],data.ohlc[i-2],data.ohlc[i-3]
        cond1 = ohlc1>ohlc2 and ohlc2>ohlc3
        cond2 = (ohlc1>ohlc2+std/2)
        cond3 = data.High[i-2]<data.Close[i-1]
        sig1 = np.sum(data.Close[i-sma:i])>np.sum(data.Close[i-sma-1:i-1]) and np.sum(data.Close[i-sma-1:i-1])>np.sum(data.Close[i-2-sma:i-2])
        lcond = (cond1 or cond2) and cond3 and sig1
        if(lcond):
            if(data.High[i]>data.Open[i]*1.005):
                pnl.append(data.Open[i]*0.005)
            else:
                pnl.append(data.Close[i]-data.Open[i])
    aa = np.array(pnl)
    return aa

In [188]:
tickers = pd.read_csv('nifties/nifty500.csv').Symbol

In [ ]:
df = []
for t in tickers:
    try:
        aa = analysis(t)
#         print(aa)
        df.append([t,aa.sum(),len(aa),len(aa[aa>0]),len(aa[aa<0])])
    except:
        print("error in ",t)

error in  360ONE
error in  AAVAS
error in  ATGL
error in  AWL
error in  ABSLAMC
error in  AETHER
error in  AFFLE
error in  ALOKINDS
error in  ANGELONE
error in  ANURAS
error in  APTUS
error in  BANDHANBNK
error in  BDL
error in  MAPMYINDIA
error in  CAMPUS
error in  CHALET
error in  CHEMPLASTS
error in  CLEAN
error in  CREDITACC
error in  DALBHARAT
error in  DELHIVERY
error in  DEVYANI
error in  EASEMYTRIP
error in  EQUITASBNK
error in  NYKAA
error in  FINEORG
error in  GRINFRA
error in  GLAND
error in  GOCOLORS
error in  GREENPANEL
error in  FLUOROCHEM
error in  HDFCAMC


In [181]:
df = pd.DataFrame(df,columns = ['Ticker','Totalpnl','trades','tradeswon','tradeslost'])

In [185]:
len(df[df.Totalpnl>0])

42

,Ticker,Totalpnl,trades,tradeswon,tradeslost
0,3MINDIA,-3846.46950,389,218,167
1,ABB,-203.87975,360,193,163
2,ACC,-39.18050,423,235,188
3,AIAENG,-171.69875,373,222,148
4,APLAPOLLO,-52.62460,357,202,152
...,...,...,...,...,...
402,ZEEL,18.85950,371,238,131
403,ZENSARTECH,-44.30640,308,172,130
404,ZYDUSLIFE,-193.50300,361,181,177
405,ZYDUSWELL,-353.19400,314,157,156
